<a href="https://colab.research.google.com/github/thor4/crypto/blob/main/Conklin_Technical_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conklin Technical Test
2/3/22

*Please write a python script which given a tx hash can identify the gainers and losers of the transaction. Who ended up making assets and who ended up losing. Please disregard any fees earned as assets gained.*

Install the web3 library

In [2]:
%pip install web3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 501.8/501.8 KB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.3/102.3 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 KB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.4/240.4 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for

Load relevant libraries.

In [11]:
from web3 import Web3
import requests
import json

Identify the Ethereum node. Here I'm using a temporary node from Infura.

In [2]:
w3 = Web3(Web3.HTTPProvider("https://mainnet.infura.io/v3/518dbfd281604e4f8dbd0678973ed9bd"))

Define a get_token_price function that pulls pricing information from dexscreener based on a token address. Unfortunately dexscreener doesn't provide historical pricing. So, we can't get the price at the time of the transaction. However, it has pricing for more tokens than competitors. The tradeoff here is between pricing data for a larger range of tokens and time-dependent pricing. The decision was to maximize the chance we get a price for any tokens involved in a transaction.

In [22]:
def get_token_price(token_address):
    response = requests.get(f"https://api.dexscreener.com/latest/dex/tokens/{token_address}").json()
    if response['pairs']:
      return float(response['pairs'][0]['priceUsd'])
    return 0

Define a parse_transaction function that takes a transaction hash and saves its details in transaction and receipt variables. It also iterates through all associated transfer events and identifies who gains and loses each time, saving the results in a dict structure. The USD-denominated running total for all gainers and losers are updated after each event and the resulting dict structures are returned at the end.

In [37]:
def parse_transaction(transaction_hash):
    transaction = w3.eth.getTransaction(transaction_hash)
    receipt = w3.eth.getTransactionReceipt(transaction_hash)
    
    gainers = {}
    losers = {}
    
    for log in receipt["logs"]:
        if log["topics"][0].hex() == "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef":
            from_address = w3.toChecksumAddress(log["topics"][1].hex()[-40:])
            to_address = w3.toChecksumAddress(log["topics"][2].hex()[-40:])
            token_address = w3.toChecksumAddress(log["address"])

            token_price = get_token_price(token_address)
            token_value = int(log["data"], 16) / 10**18
            token_usd_value = token_value * token_price

            if from_address in gainers:
                gainers[from_address] -= token_usd_value
                if gainers[from_address] < 0:
                    losers[from_address] = abs(gainers[from_address])
                    gainers.pop(from_address)
            elif from_address in losers:
                losers[from_address] += token_usd_value
                if losers[from_address] < 0:
                    gainers[from_address] = abs(losers[from_address])
                    losers.pop(from_address)
            else:
                losers[from_address] = token_usd_value

            if to_address in gainers:
                gainers[to_address] += token_usd_value
                if gainers[to_address] < 0:
                    losers[to_address] = abs(gainers[to_address])
                    gainers.pop(to_address)
            elif to_address in losers:
                losers[to_address] -= token_usd_value
                if losers[to_address] < 0:
                    gainers[to_address] = abs(losers[to_address])
                    losers.pop(to_address)
            else:
                gainers[to_address] = token_usd_value

    return gainers, losers

Define and process the transaction hash to identify its gainers and losers.

In [38]:
tx_hash = "0x6200bf5c43c214caa1177c3676293442059b4f39eb5dbae6cfd4e6ad16305668"
Gainers, Losers = parse_transaction(tx_hash)
print("Gainers:", Gainers)
print("Losers:", Losers)

Gainers: {'0xDb2d869ac23715af204093e933f5EB57F2DC12a9': 36707.81634026766}
Losers: {'0xBA12222222228d8Ba445958a75a0704d566BF2C8': 0.0, '0xb835752Feb00c278484c464b697e03b03C53E11B': 36707.8163402621}


This tells us that the malicious contract (0xDb2d869ac23715af204093e933f5EB57F2DC12a9) gained about $36,707.82.

The Balancer deployer responsible for the flash loan (0xBA12222222228d8Ba445958a75a0704d566BF2C8) was flat at $0 since the loan was completely paid back.

The Uniswap LP contract address for the WETH-TINU pair (Uniswap V2: TINU 12, 0xb835752Feb00c278484c464b697e03b03C53E11B) lost the most at $36,707.82

This nets out nicely.

## Improvements for future consideration:


1.   Capture final withdrawals event(s) to identify which address ended up with potential profit.
2.   Identify addresses who held positions in the Uniswap contract address that were exposed to the losses and calculate their realized losses.
3.   Move $0 balances to a new dict for neutral parties that neither gained nor lost on the transaction.

